## Notebook to along side lecture 2 of cs20si 
### Tensorflow for deep learning research

_Note that graph summaries will not be available while the notebook is serving from the directory. So tensorboard will not see the summaries. Killing the nodebook server only way I know of for seeing the tensorboard graph. I assume that the server is creating a lock on the resource._

- tf.mul is deprecated if tf1.0, replaced by tf.multiply

In [54]:
import tensorflow as tf
tf.__version__

'1.0.1'

#### TF
- Phase 1 : Assemble graph
- Phase 2 : Use session to run the operations in the graph

In [3]:
a = tf.add(2,3)
with tf.Session() as sess:
    print(sess.run(a))

##### Graph primitives
- Nodes : Operators, variables and constants
- Edges : Tensors

_A Session object encapsulates the environment in which Operations 
objects are executed and Tensor objects are evaluated_

In [7]:
x = 2
y = 3
op1 = tf.add(x, y)

#
# note the tf.mul is elementwise
op2 = tf.multiply(x, y)
op3 = tf.pow(op1, op2)
with tf.Session() as sess:
    op3 = sess.run(op3)
    


##### Summaries | Tensorboard

In [55]:
a = tf.constant(2)
b = tf.constant(3)

x = tf.add(a,b)
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graph', sess.graph)
    print(sess.run(x))
writer.close()

5


##### Constants

In [56]:
a  = tf.constant([2, 2], name="a")
b  = tf.constant([[0, 1], [2, 3]], name="b")

x = tf.add(a, b, name="add")
y = tf.multiply(a, b, name="mul")

with tf.Session() as sess:
    x, y = sess.run([x, y])
    print x

[[2 3]
 [4 5]]


In [57]:
a = tf.zeros([2, 3], tf.int32, name="zeros")

# creates a tensor filled with a scalar value

# in numpy takes two steps 
# create Array a and then a.fill

# fill a 2, 3 Tensor with 8's
b = tf.fill([2, 3], 8)

# what does the optimize parameter do?
ones = tf.ones_like(a, optimize=True)

with tf.Session() as sess:
    print(sess.run([a, ones]))

[array([[0, 0, 0],
       [0, 0, 0]], dtype=int32), array([[1, 1, 1],
       [1, 1, 1]], dtype=int32)]


In [12]:
# create 10 values from 0.0 and 1.0
# note: can't pass int types to linspace start parameter
# i.e tf.linspace(0, 1, 10 ) will raise an attribute error

# but tf.linspace(0., 1, 10) will cast the end stop parameter to float.

a = tf.linspace(0., 1, 10)
b = tf.range(0, 10, 2) # creates [0, 2, 4, 6, 8]

with tf.Session() as sess:
    print(sess.run([a, b]))

[array([ 0.        ,  0.11111111,  0.22222222,  0.33333334,  0.44444445,
        0.55555558,  0.66666669,  0.77777779,  0.8888889 ,  1.        ], dtype=float32), array([0, 2, 4, 6, 8], dtype=int32)]


In [59]:
shape = [3, 3]
a = tf.random_normal(shape, mean=0.0, stddev=1.0, 
                     dtype=tf.float32, seed=None)

# sample from a normal distribution with specified mean and stddev
# however, values greater than 2 standard deviations from the mean are
# discarded and repicked

b = tf.truncated_normal(shape, mean=0.0, stddev=1.0, 
                     dtype=tf.float32, seed=None)

 

# shuff = tf.random_shuffle(tf.mul(a, b), seed=None, name=None)

# randomly crop a [2 2] matrix from `a`

# let's create the size parameter  for tf.crop
# for more info do tf.random_crop? in a cell
# `size` needs to be a 1-D vector with size the rank of `a`

# crop a 2, 2 random matrix from a
size = tf.fill([tf.rank(a)], 2)

crop  = tf.random_crop(a, size, seed=None, name=None)
# multi = tf.multinomial(logits, num_samples, seed=None, name=None)
# gamm = tf.random_gamma(shape, .3 , beta=None, dtype=tf.float32,
#                         seed=None, name=None)

with tf.Session() as sess:
#     print(sess.run([a, b]))
    print(sess.run(crop))
    


[[-0.31536332  0.70742536]
 [ 0.57206273  1.48380411]]


##### Operations

In [60]:
a = tf.constant([3, 6])
b = tf.constant([2, 2])

addn = tf.add_n([a, b, b]) # >> [7 10]. Equivalent to a + b + b

# tf.matmul is matrix multiplication so dimension must confirm
# tf.matmul(a, b) # >> ValueError

# reshape a as a row vector and reshape b to a column vector
# now matrix multiplication is possible.

matmul = tf.matmul(tf.reshape(a, [1, 2]), tf.reshape(b, [2, 1])) # >> [[18]]
divide = tf.div(a, b) # >> [1 3]
mod = tf.mod(a, b) # >> [1 0]

with tf.Session() as sess:
#     print(sess.run([a, b]))
    print(sess.run(matmul))
#     print(sess.run(b))


[[18]]


##### Variables

In [61]:
a = tf.Variable(2, name="scaler")
# create variable b as a vector
b = tf.Variable([2, 3], name="vector")
# create variable c as a 2x2 matrix
c = tf.Variable([[0, 1], [2, 3]], name="matrix")
# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))

# must initialize variables
# here we do them all at one
init = tf.global_variables_initializer()

# initialize only a subset of variables
# init = tf.global_variable_initializer([a, b], name="init_ab")

with tf.Session() as sess:
    sess.run(init)

# initialize a single variable 
# with tf.Session() as sess:
#     sess.run(W.initializer)

In [62]:
# W is a random 700 x 100 variable object
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print W


Tensor("Variable_10/read:0", shape=(700, 10), dtype=float32)


##### .eval() 

In [63]:
# W is a random 700 x 100 variable object
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print W.eval()

[[-1.22230804 -0.27763152  0.29708382 ..., -0.42395937  0.15295221
  -1.82301545]
 [-1.4102329   0.24177302  1.01148081 ...,  1.5404228  -0.8029266
   1.22561824]
 [-0.12743907 -0.34546509 -1.3957969  ...,  0.9137609  -0.03999023
  -0.07049976]
 ..., 
 [-1.92618167 -0.59211665  0.13615945 ...,  1.08274746  0.73251981
   1.25107837]
 [ 0.67549735  0.35665256 -1.28448474 ..., -0.20284112  0.59224588
  -0.73523259]
 [ 0.84100056  0.18613793  0.3852838  ..., -1.06514704 -0.35162124
  -0.93291306]]


##### tf.Varaible.assign()


In [64]:
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(assign_op)
    print(W.eval())

100


In [65]:
# create a variable whose original value is 2
my_var = tf.Variable(2, name="my_var")

# assign a * 2 to a and call that op a_times_two
my_var_times_two = my_var.assign(2 * my_var)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    sess.run(my_var_times_two) # >> 4
    sess.run(my_var_times_two) # >> ?????


In [66]:
my_var = tf.Variable(10)

with tf.Session() as sess:
    sess.run(my_var.initializer)
    # increment by 10
    sess.run(my_var.assign_add(10)) # >> 20
    # decrement by 2
    sess.run(my_var.assign_sub(2))

##### Variable persistence  in sessions

In [25]:
W = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
sess2.run(W.initializer)

print sess1.run(W.assign_add(10)) # >> 20
print sess2.run(W.assign_sub(2)) # >> 8

print sess1.run(W.assign_add(100)) # >> 120
print sess2.run(W.assign_sub(50)) # >> -42

sess1.close()
sess2.close()

20
8
120
-42


##### Interactive sessions
- sets itself as default session

In [27]:
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)

c = a * b
print(c.eval())
sess.close()


30.0


##### Control Dependencies
- sets up order of operations
- could be nested

In [35]:
g = tf.Graph() 
with g.as_default():
    a = tf.constant(5.0)
    b = tf.constant(6.0)
    c = a * b
    
    with g.control_dependencies([a, b]):
        d = a + c

##### placeholders 
- feed values to placeholder using dicitonaries
- placeholders are valid operations

In [39]:
# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])

# create a constant of type float 32-bit, shape is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b # Short for tf.add(a, b)b

with tf.Session() as sess:
    # print sess.run(c) # Error because a doesn’t have any value
    # tensor `a` is a key not the string 'a'
    print(sess.run(c, {a: [1, 2, 3]}))

[ 6.  7.  8.]


In [41]:
# create operations, tensors, etc (using the default graph)
a = tf.add(2, 5)
b = tf.multiply(a, 3)

with tf.Session() as sess:
    # define a dictionary that says to replace the value of 'a' with 15
    replace_dict = {a: 15}
    # Run the session, passing in 'replace_dict' as the value to 'feed_dict'
    print(sess.run(b, feed_dict=replace_dict)) # returns 45

45


##### Normal loading
- create node for 'add node' before executing graph
- 'add node' is added once

In [51]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y) # you create the node for add node before executing the graph

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph/l2', sess.graph)
    for _ in range(10):
        print(sess.run(z))
        writer.close()
        
#     print(tf.get_default_graph().as_graph_def())

30
30
30
30
30
30
30
30
30
30


##### Lazy loading example 
- add node is not added to the graph
- does not show up in the tensorboard
- not a bug though
- node 'Add' is added n number of times
- leads to bloated, expensive to load. pass around graph
-

In [43]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph/l2', sess.graph)
    for _ in range(10):
        print(sess.run(tf.add(x, y))) # someone decides to be clever to save one line of code
        writer.close()

30
30
30
30
30
30
30
30
30
30
